In [1]:
import numpy as np
import pyvista as pv
from tqdm import tqdm

from kale import Engine, save_movie, show_ui
from kale.algorithms import (
    contour_banded,
    subdivide_algorithm,
    scalars_operation_algorithm,
)

In [2]:
# Requires PyVista 0.38.1
pv.set_jupyter_backend("server")
pv.global_theme.trame.interactive_ratio = 2
pv.global_theme.trame.still_ratio = 2
pv.global_theme.image_scale = 2  # upscales the saved screenshots/video frames

In [3]:
# MESH_GEOMETRY_FILE_NAME = "nankai_very_very_dense.vtk"
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_03_16_15_24.hdf"
MESH_Z_SCALE = 0.01
TIME_STEP_INTERESTING = 55849
TIME_STEP_INTERESTING = 1000

LINE_WIDTH = 1
CONTOUR_LINE_COLOR = "black"
COLOR_MAPS = {
    "cumulative_slip": "CET_L19",  #'CET_C3',
    "geometric_moment": "CET_R3",
    "last_event_slip": "CET_L19",
    "loading_rate": "CET_C3",
}

# TODO: The same fontsizes seem to produce different sized fonts on the figure
# I'd like all fonts to be the same size and type
COLORBAR_FONT_SIZE = 34
AXES_FONT_SIZE = 26
TIMESTEP_FONT_SIZE = 12

SCALAR_BAR_OPTS = dict(
    # Position (coordinates are on the viewport)
    vertical=True,
    height=0.20,
    width=0.05,
    position_x=0.05,
    position_y=0.40,
    # Labels
    title_font_size=AXES_FONT_SIZE,
    label_font_size=AXES_FONT_SIZE,
    n_labels=3,
    italic=False,
    fmt="%.1f",
    font_family="arial",
    shadow=True,
)

## Load Data

Set up the driver of the data IO and time step management with the `kale.Engine` class

In [4]:
engine = Engine(MESH_GEOMETRY_FILE_NAME, MESH_VALUES_FILE_NAME, zscale=MESH_Z_SCALE)
engine.keys

['cumulative_slip', 'geometric_moment', 'last_event_slip', 'loading_rate']

In [5]:
# Set which array to be active
engine.mesh.active_scalars_name = "geometric_moment"
engine.time_step = TIME_STEP_INTERESTING

In [6]:
cpos = [
    (246.72654935831832, 56.640050977986135, 19.47264855846906),
    (235.64372937685064, 45.67443514953468, -0.9481135945038834),
    (-0.6591979033815945, -0.45237409343002455, 0.6006794517633125),
]


## Plot with time controls

In [7]:
pl = pv.Plotter()
pl.add_mesh(
    engine.algorithm,
    cmap=COLOR_MAPS[engine.mesh.active_scalars_name],
    clim=engine.clim(),
    scalar_bar_args=SCALAR_BAR_OPTS,
)
pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
# pl.camera_position = cpos
show_ui(engine, pl)

In [ ]:
# Run cell to fetch current position
pl.camera_position

## Banded Contouring

In [8]:
# TODO: How do I maniputlate the quantity geometic moment here?
# There's a lot of experimenting to do in order to see the best way to plot things.
# For example I might like to take the cube root of this or the log or sigmoid.
engine.mesh.active_scalars_name = "geometric_moment"

# Manipulate the quantity
operation = np.cbrt
MANIPULATED_ARRAY_NAME = f"cbrt({engine.mesh.active_scalars_name})"
manipulated = scalars_operation_algorithm(
    engine.algorithm, operation, output_scalars_name=MANIPULATED_ARRAY_NAME
)

# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-2500, 2500, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

contour, edges = contour_banded(
    manipulated,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
)

pl = pv.Plotter()
pl.add_mesh(engine.boundary, color="k", line_width=LINE_WIDTH)
pl.add_mesh(
    contour,
    cmap=COLOR_MAPS[engine.mesh.active_scalars_name],
    clim=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
    n_colors=N_COLORS,
    # scalar_bar_args=dict(title=f"{MANIPULATED_ARRAY_NAME}", **SCALAR_BAR_OPTS),
    scalar_bar_args=dict(title=f"m", **SCALAR_BAR_OPTS),
)
pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
pl.add_text(f"i = {1:05d}", name="time-step-label", font_size=TIMESTEP_FONT_SIZE)

box = pv.Box(engine.mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
# pl.camera_position = cpos
pl.camera.zoom(1.2)

# Drawing lines as a grid proxy
# mesh = pv.Line((0, 0, 0), (0, 0, 1))
# mesh.plot(color='k', line_width=10)

# Experimenting with lights
light = pv.Light((-2, 2, 0), (0, 0, 0), "white")
pl.add_light(light)
show_ui(engine, pl)


### 4-panel view (all variables)

This still needs some optimization to make look really nice but here is a start

In [ ]:
engine.keys

In [ ]:
pl = pv.Plotter(shape=(1, 4), window_size=(1024 * 2, 768))

for i, var in enumerate(engine.keys):
    pl.subplot(0, i)
    contour, edges = contour_banded(
        # TODO: set contour levels for each scalar, for now use N
        engine.algorithm,
        11,
        rng=engine.clim(var),
        scalars=var,
    )
    pl.add_mesh(
        contour,
        cmap=COLOR_MAPS[var],
        clim=engine.clim(var),
        n_colors=11 - 1,
        scalar_bar_args=SCALAR_BAR_OPTS,
    )
    pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
    pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)

pl.link_views()
pl.camera_position = cpos
show_ui(engine, pl)

## Save Videos

In [ ]:
# var = "cumulative_slip"
var = "geometric_moment"

pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed, color="k", line_width=LINE_WIDTH)
contour, edges = contour_banded(
    engine.algorithm,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=var,
)
pl.add_mesh(
    contour,
    cmap=COLOR_MAPS[var],
    clim=CLIM,
    n_colors=N_COLORS,
    scalar_bar_args=SCALAR_BAR_OPTS,
)
pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)

pl.camera_position = cpos

pl.image_scale = 2
# Make a video over time steps
# save_movie(engine, plotter, f'{var}.mpg', framerate=24, quality=1)
step = 500
# step = 1
pl.open_movie(f"{var}.mp4", framerate=48, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(f"time step: {tindex:05d}", name="time-step-label", font_size=18)
    pl.write_frame()
pl.close()

# Updated video style

In [14]:
# var = "cumulative_slip"
var = "geometric_moment"

engine.mesh.active_scalars_name = "geometric_moment"

# Manipulate the quantity
operation = np.cbrt
MANIPULATED_ARRAY_NAME = f"cbrt({engine.mesh.active_scalars_name})"
manipulated = scalars_operation_algorithm(
    engine.algorithm, operation, output_scalars_name=MANIPULATED_ARRAY_NAME
)

# CONTOUR_LEVELS = np.linspace(-10e9, 10e9, 11)
CONTOUR_LEVELS = np.linspace(-500, 500, 11)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]

pl = pv.Plotter()
pl.add_mesh(engine.boundary_smoothed, color="k", line_width=LINE_WIDTH)
# contour, edges = contour_banded(
#     engine.algorithm,
#     CONTOUR_LEVELS,
#     rng=CLIM,
#     scalars=var,
# )

contour, edges = contour_banded(
    manipulated,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars=MANIPULATED_ARRAY_NAME,
)

pl.add_mesh(
    contour,
    cmap=COLOR_MAPS[var],
    clim=CLIM,
    n_colors=N_COLORS,
    # scalar_bar_args=SCALAR_BAR_OPTS,
    scalar_bar_args=dict(title=f"m", **SCALAR_BAR_OPTS),
)
pl.add_mesh(edges.GetOutputPort(1), color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)
pl.add_mesh(engine.boundary, color=CONTOUR_LINE_COLOR, line_width=LINE_WIDTH)

box = pv.Box(engine.mesh.bounds)
pl.add_mesh(box, color="lightgrey", culling="front")
# pl.camera_position = cpos
# pl.camera.zoom(1.0)
pl.image_scale = 2

# Make a video over time steps
# save_movie(engine, plotter, f'{var}.mpg', framerate=24, quality=1)
step = 100
# step = 1
pl.open_movie(f"{var}.mp4", framerate=60, quality=5)
for tindex in tqdm(range(0, engine.max_time_step, step)):
    engine.time_step = tindex
    pl.add_text(f"i = {tindex:05d}", name="time-step-label")
    pl.write_frame()
pl.close()


100%|██████████| 200/200 [00:28<00:00,  6.93it/s]
